In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
import numpy as np
from mlagents_envs.environment import ActionTuple


In [2]:
# initialise envs

# env =  UE(file_name="stage0_160523\stage0_copy",seed=1,side_channels=[])
file_name_train = "C:\\Users\\Palaash.HPZ\\Desktop\\RL-concept-learning_large_build_envs\\build_envs\\windows\\S2 180723\\build"
file_name_test = "C:\\Users\\Palaash.HPZ\\Desktop\\RL-concept-learning_large_build_envs\\build_envs\\windows\\S2_test 180723\\build"
env_train =  UE(file_name=file_name_train,seed=1,side_channels=[],worker_id=8,no_graphics = False)
env_train.reset()
env_test =  UE(file_name=file_name_test,seed=1,side_channels=[],worker_id=9,no_graphics = False)
env_test.reset()

In [3]:
# env_train.close()
# env_test.close()

In [4]:
# model
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

vision_output_dim = 3136
num_words = 35  # Number of unique words in the vocabulary
language_output_dim = 128
embedding_dim = 128
mixing_dim = 256
lstm_hidden_dim = 256
num_actions = 4

# (3,128,128) --> (64,7,7) = 3136 (3-layer CNN)
class VisualModule(nn.Module): 
    def __init__(self):
        super(VisualModule, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=3, padding=0),
            nn.ReLU()
        )
        # self.conv = nn.Sequential(
        #     nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(128, 64, kernel_size=5, stride=2, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        # )

    def forward(self, vt):
        encoded_vt = self.conv(vt)
        return encoded_vt.view(vt.size(0), -1).squeeze()

# one-hot encoding [0 0 1 0 0] --> 128 dimensional embedding (FF)
# S1:5 S2:5 S3:11 S4:9 --> 30 + 5 (noun) = 35 in total
class LanguageModule(nn.Module): 
    def __init__(self, num_words, embedding_dim):
        super(LanguageModule, self).__init__()
        self.embedding = nn.Linear(num_words, embedding_dim)

    def forward(self, lt):
        embedded_lt = self.embedding(lt)
        return embedded_lt

# 3136(vision) + 128 (language) --> 256 dimensional embedding (FF)
class MixingModule(nn.Module):
    def __init__(self, vision_output_dim, language_output_dim, mixing_dim):
        super(MixingModule, self).__init__()
        self.linear = nn.Linear(vision_output_dim + language_output_dim, mixing_dim)

    def forward(self, vision_output, language_output):
        combined_output = torch.cat((vision_output, language_output), dim=0)
        mixed_output = self.linear(combined_output)
        return mixed_output

class LSTMModule(nn.Module):
    def __init__(self,mixing_dim,lstm_hidden_dim):
        super(LSTMModule, self).__init__()
        self.lstm = nn.LSTMCell(mixing_dim, lstm_hidden_dim)
    
    def forward(self,mixed_output,lstm_hidden_state):
        lstm_hidden_state = self.lstm(mixed_output, lstm_hidden_state) 
        # lstm_output = lstm_hidden_state[0] # output is (hidden_state,cell_state), we need hidden state, shape (1,256)
        return lstm_hidden_state

class Agent(nn.Module):
    def __init__(self, num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions):
        super(Agent, self).__init__()
        self.language_module = LanguageModule(num_words, embedding_dim)
        self.visual_module = VisualModule()
        self.mixing_module = MixingModule(vision_output_dim, language_output_dim, mixing_dim)
        self.lstm_module = LSTMModule(mixing_dim, lstm_hidden_dim)
        self.action_predictor = nn.Linear(lstm_hidden_dim, num_actions)
        self.value_estimator = nn.Linear(lstm_hidden_dim, 1)

    def forward(self, vt, lt, lstm_hidden_state):
        vision_output = self.visual_module(vt)
        language_output = self.language_module(lt)
        mixed_output = self.mixing_module(vision_output, language_output).unsqueeze(0)
        lstm_output = self.lstm_module(mixed_output,lstm_hidden_state)
        action_probs = self.action_predictor(lstm_output[0]) 
        value_estimate = self.value_estimator(lstm_output[0])
        return action_probs,value_estimate,lstm_output
        
        
    def save(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        if not os.path.exists(path):
            os.makedirs(path)
        torch.save(self.state_dict(), os.path.join(path, f'agent_{episode}.pt'))

    def load(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        self.load_state_dict(torch.load(os.path.join(path, f'agent_{episode}.pt')))    

        


C:\Users\Palaash.HPZ\anaconda3\envs\transformers_mlagents\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# function to test model on test env
def test(agent,test_episode,test_episode_reward,test_average_reward,test_steps,test_actor_loss,test_critic_loss,test_entropy_loss,test_total_loss):
    env = env_test
    TEST_EPISODES = 100
    tracked_agent = -1
    entropy_term = 0
    for episode in range(TEST_EPISODES):
        test_episode += 1
        t0 = time.time()
        episode_reward = 0
        # env.reset()
        behavior_name=list(env.behavior_specs)[0]
        spec=env.behavior_specs[behavior_name]
        # state = env.reset().astype(np.float32)
        STEPS = 0
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        # state -- vt, lt, lstm
        vt = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
        index1 = int(decision_steps.obs[1][0][0])
        index2 = int(decision_steps.obs[1][0][1])+5
        print(f'TEST: ---{hashmap[index2]} {hashmap[index1]}---')
        # 0-capsule,1-cube,2-cylinder,3-prism,4-sphere 
        lt = torch.zeros(35).to(device)
        lt[index1],lt[index2] = 1,1
        lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
        done = False
        while True:

            # Need to use when calculating the loss
            log_probs = []
            # values = []
            values = torch.empty(0).to(device)
            rewards = []

            
            lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
            STEPS += 1
            policy_dist, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state)
            # value = value.detach()
            dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
            

            action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
            # action_dist = Categorical(F.softmax(policy_dist,dim=1))
            action = action_dist.sample() # sample an action from action_dist
            action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()
            
            log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
            # log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
            # entropy = -np.sum(np.mean(dist)* np.log(dist))
            entropy = F.cross_entropy(policy_dist.detach(), action)

            discrete_actions = np.array(action_onehot).reshape(1,4)*speed
            action_tuple = ActionTuple()
            action_tuple.add_discrete(discrete_actions)
            env.set_actions(behavior_name,action_tuple)
            env.step()
            decision_steps, terminal_steps = env.get_steps(behavior_name)

            if tracked_agent == -1 and len(decision_steps) >= 1:
                tracked_agent = decision_steps.agent_id[0]
                # print(tracked_agent)

            if tracked_agent in terminal_steps: # roll over or hit the target
                print('TEST: Agent in terminal steps')
                done = True
                reward = terminal_steps[tracked_agent].reward
                if reward > 0:
                    pass
                else: reward = -1 # roll over or other unseen conditions

                print(f'TEST: Terminal Step reward: {reward}')

            elif tracked_agent in decision_steps: # the agent which requires action
                reward = decision_steps[tracked_agent].reward
                # print(f'Decision Step reward: {reward}')
                if reward<0:
                    print(f'TEST: Decision Step reward: {reward}')
            if STEPS >= MAX_STEPS:
                reward = -10
                print(f'TEST: Max Step Reward: {reward}')
                env.reset()
                done = True
            if STEPS % 100 == 0:
                print (f'TEST: Step: {STEPS}')

            episode_reward = episode_reward + reward

            rewards.append(reward)
            # values.append(value)
            values = torch.cat((values, value), dim=0)
            log_probs.append(log_prob)
            entropy_term = entropy_term + entropy
            vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
            vt = vt_new

            if done:
                # _, Qval,_ = agent(vt_new,lt,lstm_hidden_state)
                # Qval = Qval.detach()
                break
            
            
        discounted_rewards = np.zeros_like(values.cpu().detach().numpy())
        cumulative = 0
        for t in reversed(range(len(rewards))):
            cumulative = rewards[t] + LAM * cumulative # Monte Carlo
            discounted_rewards[t] = cumulative
        # print(f'rewards:{rewards}, discounted_rewards:{discounted_rewards}')
        # Advantage Actor Critic

        # Qvals[-1] = rewards[t] + LAM * Qval      or       Qvals[-1] = rewards[t]                   
        # for t in range(len(rewards)-1):
        #         Qvals[t] = rewards[t] + LAM * values[t+1]
        
        # r_(t+1) = R(s_t|a_t)--> reward[t]        a_t, V_t = agent(s_t)
        # A_t = r_(t+1) + LAM * V_(t+1) - V_t 
        #     = Q_t - V_t
        
        # Monte Carlo Advantage = reward + LAM * cumulative_reward
        # Actor_loss = -log(pai(s_t|a_t))*A_t
        # Critic_loss = A_t.pow(2) *0.5
        # Entropy_loss = -F.entropy(pai(St),index) * 0.001

        # entropy = -np.sum(np.mean(dist) * np.log(dist))
        
        #update actor critic
        
        # values = torch.FloatTensor(values).requires_grad_(True).to(device)
        discounted_rewards = torch.FloatTensor(discounted_rewards.astype(np.float32)).to(device)
        log_probs = torch.stack(log_probs)
        advantage = discounted_rewards - values
        actor_loss = (-log_probs * advantage).mean()
        critic_loss = 0.5 * torch.square(advantage).mean()
        entropy_term /= STEPS
        entropy_loss = -0.1 * entropy_term
        ac_loss = actor_loss + critic_loss + entropy_loss
        test_episode_reward.append(float(episode_reward))
        test_steps.append(STEPS)
        test_actor_loss.append(float(actor_loss))
        test_critic_loss.append(float(critic_loss))
        test_entropy_loss.append(float(entropy_loss))
        test_total_loss.append(float(ac_loss))

        if test_episode >= 100:
            avg_score = np.mean(test_episode_reward[-100:])
            test_average_reward.append(avg_score)
            print('Testing  | Episode: {}/{}  | Episode Reward: {:.0f}  | Average Reward {:.2f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                .format(episode + 1, TEST_EPISODES, episode_reward, avg_score, actor_loss, critic_loss,entropy_loss,  ac_loss, STEPS))
        else:  print('Testing  | Episode: {}/{}  | Episode Reward: {:.0f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                .format(episode + 1, TEST_EPISODES, episode_reward, actor_loss, critic_loss, entropy_loss,  ac_loss, STEPS))
    return test_episode,test_episode_reward,test_average_reward,test_steps,test_actor_loss,test_critic_loss,test_entropy_loss,test_total_loss
                

In [ ]:
# training loop
import argparse
import time
import json
# import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

entropy_term = 0
# add arguments in command --train/test
# parser = argparse.ArgumentParser(description='Train or test neural net motor controller.')
# parser.add_argument('--train', dest='train', action='store_true', default=False)
# parser.add_argument('--test', dest='test', action='store_true', default=True)
# args = parser.parse_args()
train = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 

S0_ALG_NAME = 'S2'
S0_ENV_ID = '22'
S0_episode = 48000

ALG_NAME = 'S2'
ENV_ID = '25'
TRAIN_EPISODES = 68000  # number of overall episodes for training  # number of overall episodes for testing
MAX_STEPS = 500  # maximum time step in one episode
LAM = 0.95  # reward discount in TD error
lr = 2.5e-5  #0.00005 
speed = 3
num_steps = 250 # the step for updating the network
test_episode = 0
if __name__ == '__main__':
    agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
    agent.load(S0_episode,S0_ALG_NAME,S0_ENV_ID)
    agent.to(device)
    optimizer = optim.RMSprop(agent.parameters(), lr=lr)
    best_score = float('-inf')
    hashmap = {
        0: 'capsule',
        1: 'cube',
        2: 'cylinder',
        3: 'prism',
        4: 'sphere',
        5: 'red',
        6: 'green',
        7: 'blue',
        8: 'yellow',
        9: 'black'}
    if train:
        entropy_term = 0
        test_episode_reward = []
        test_average_reward = []
        test_steps = []
        test_actor_loss = []
        test_critic_loss = []
        test_entropy_loss = []
        test_total_loss = []
        tracked_agent = -1
        test_episode = 0
        all_episode_reward = []
        all_average_reward = []
        all_steps = []
        all_actor_loss = []
        all_critic_loss = []
        all_entropy_loss = []
        all_total_loss = []
        env = env_train
        for episode in range(TRAIN_EPISODES):
            t0 = time.time()
            episode_reward = 0
            # env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]
            # state = env.reset().astype(np.float32)
            STEPS = 0

            decision_steps, terminal_steps = env.get_steps(behavior_name)
            # state -- vt, lt, lstm
            vt = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
            index1 = int(decision_steps.obs[1][0][0])
            index2 = int(decision_steps.obs[1][0][1])+5
            print()
            print(f'---{hashmap[index2]} {hashmap[index1]}---')
            # 0-capsule,1-cube,2-cylinder,3-prism,4-sphere 
            lt = torch.zeros(35).to(device)
            lt[index1],lt[index2] = 1,1
            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while True:

                # Need to use when calculating the loss
                log_probs = []
                # values = []
                values = torch.empty(0).to(device)
                rewards = []

                for steps in range(num_steps):
                    lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
                    STEPS += 1
                    policy_dist, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state)
                    # value = value.detach()
                    dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
                    

                    action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
                    # action_dist = Categorical(F.softmax(policy_dist,dim=1))
                    action = action_dist.sample() # sample an action from action_dist
                    action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()
                    
                    log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # entropy = -np.sum(np.mean(dist)* np.log(dist))
                    entropy = F.cross_entropy(policy_dist.detach(), action)

                    discrete_actions = np.array(action_onehot).reshape(1,4)*speed
                    action_tuple = ActionTuple()
                    action_tuple.add_discrete(discrete_actions)
                    env.set_actions(behavior_name,action_tuple)
                    env.step()
                    decision_steps, terminal_steps = env.get_steps(behavior_name)

                    if tracked_agent == -1 and len(decision_steps) >= 1:
                        tracked_agent = decision_steps.agent_id[0]
                        # print(tracked_agent)

                    if tracked_agent in terminal_steps: # roll over or hit the target
                        print('Agent in terminal steps')
                        done = True
                        reward = terminal_steps[tracked_agent].reward
                        if reward > 0:
                            pass
                        else: reward = -1 # roll over or other unseen conditions

                        print(f'Terminal Step reward: {reward}')

                    elif tracked_agent in decision_steps: # the agent which requires action
                        reward = decision_steps[tracked_agent].reward
                        # print(f'Decision Step reward: {reward}')
                        if reward<0:
                            print(f'Decision Step reward: {reward}')
                    if STEPS >= MAX_STEPS:
                        reward = -10
                        print(f'Max Step Reward: {reward}')
                        env.reset()
                        done = True
                    if STEPS % num_steps == 0:
                        print (f'Step: {STEPS}')

                    episode_reward = episode_reward + reward

                    rewards.append(reward)
                    # values.append(value)
                    values = torch.cat((values, value), dim=0)
                    log_probs.append(log_prob)
                    entropy_term = entropy_term + entropy
                    vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                    vt = vt_new

                    if done or steps == num_steps-1:
                        # _, Qval,_ = agent(vt_new,lt,lstm_hidden_state)
                        # Qval = Qval.detach()
                        break
                
                
                discounted_rewards = np.zeros_like(values.cpu().detach().numpy())
                cumulative = 0
                for t in reversed(range(len(rewards))):
                    cumulative = rewards[t] + LAM * cumulative # Monte Carlo
                    discounted_rewards[t] = cumulative
                # print(f'rewards:{rewards}, discounted_rewards:{discounted_rewards}')
                # Advantage Actor Critic

                # Qvals[-1] = rewards[t] + LAM * Qval      or       Qvals[-1] = rewards[t]                   
                # for t in range(len(rewards)-1):
                #         Qvals[t] = rewards[t] + LAM * values[t+1]
                
                # r_(t+1) = R(s_t|a_t)--> reward[t]        a_t, V_t = agent(s_t)
                # A_t = r_(t+1) + LAM * V_(t+1) - V_t 
                #     = Q_t - V_t
                
                # Monte Carlo Advantage = reward + LAM * cumulative_reward
                # Actor_loss = -log(pai(s_t|a_t))*A_t
                # Critic_loss = A_t.pow(2) *0.5
                # Entropy_loss = -F.entropy(pai(St),index) * 0.001

                # entropy = -np.sum(np.mean(dist) * np.log(dist))
                
                #update actor critic
                
                # values = torch.FloatTensor(values).requires_grad_(True).to(device)
                discounted_rewards = torch.FloatTensor(discounted_rewards.astype(np.float32)).to(device)
                log_probs = torch.stack(log_probs)
                advantage = discounted_rewards - values
                actor_loss = (-log_probs * advantage).mean()
                critic_loss = 0.5 * torch.square(advantage).mean()
                entropy_term /= num_steps
                entropy_loss = -0.1 * entropy_term
                ac_loss = actor_loss + critic_loss + entropy_loss
                # ac_loss = values.mean()
                optimizer.zero_grad()
                ac_loss.backward()
                optimizer.step()
                # for name, param in agent.named_parameters():
                #     if param.grad is not None:
                #         print(name, param.grad)
                #     else:
                #         print(name, "gradients not computed")
                # for name, param in agent.named_parameters():
                #     if name == 'value_estimator.weight':
                #         print(name, param)
                
                
                if done: break


            all_episode_reward.append(float(episode_reward))
            all_steps.append(STEPS)
            all_actor_loss.append(float(actor_loss))
            all_critic_loss.append(float(critic_loss))
            all_entropy_loss.append(float(entropy_loss))
            all_total_loss.append(float(ac_loss))
            if episode >= 100:
                avg_score = np.mean(all_episode_reward[-100:])
                all_average_reward.append(avg_score)
                if avg_score > best_score:
                    best_score = avg_score
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print(f'-----The best score for averaging previous 100 episode reward is {best_score}. Model has been saved-----')
                print('Training  | Episode: {}/{}  | Episode Reward: {:.0f}  | Average Reward {:.2f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, avg_score, actor_loss, critic_loss,entropy_loss,  ac_loss, STEPS))
            else:  print('Training  | Episode: {}/{}  | Episode Reward: {:.0f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, actor_loss, critic_loss, entropy_loss,  ac_loss, STEPS))
            if episode%500 == 0:
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print("Model has been saved")
            if episode%100 == 0:
                test_episode,test_episode_reward,test_average_reward,test_steps,test_actor_loss,test_critic_loss,test_entropy_loss,test_total_loss = test(agent,test_episode,test_episode_reward,test_average_reward,test_steps,test_actor_loss,test_critic_loss,test_entropy_loss,test_total_loss)

        print(all_average_reward)
        agent.save(episode ,ALG_NAME, ENV_ID)
        print("Model has been saved")

        data = {
                    'all_average_reward': all_average_reward,
                    'all_episode_reward': all_episode_reward,
                    'all_actor_loss': all_actor_loss,
                    'all_critic_loss': all_critic_loss,
                    'all_entropy_loss': all_entropy_loss,
                    'all_total_loss': all_total_loss,
                    'all_steps': all_steps,
                } 
        file_path = f'result/{ALG_NAME}_{ENV_ID}_train.txt'
        with open(file_path, 'w') as file:
            json.dump(data, file)
        
        test_data = {
                    'all_average_reward': test_average_reward,
                    'all_episode_reward': test_episode_reward,
                    'all_actor_loss': test_actor_loss,
                    'all_critic_loss': test_critic_loss,
                    'all_entropy_loss': test_entropy_loss,
                    'all_total_loss': test_total_loss,
                    'all_steps': test_steps,
                } 
        file_path = f'result/{ALG_NAME}_{ENV_ID}_test.txt'
        with open(file_path, 'w') as file:
            json.dump(test_data, file)



cuda

---blue prism---


C:\Users\Palaash.HPZ\anaconda3\envs\transformers_mlagents\lib\site-packages\ipykernel_launcher.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1/68000  | Episode Reward: 10  | Actor loss: 0.00 | Critic loss: 0.00 | Entropy loss: -0.0000  | Total Loss: 0.00 | Total Steps: 6
Model has been saved
TEST: ---black sphere---


C:\Users\Palaash.HPZ\anaconda3\envs\transformers_mlagents\lib\site-packages\ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 1/100  | Episode Reward: 10  | Actor loss: -0.00 | Critic loss: 0.02 | Entropy loss: -0.0001  | Total Loss: 0.02 | Total Steps: 6
TEST: ---blue cylinder---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 2/100  | Episode Reward: 10  | Actor loss: 0.04 | Critic loss: 4.35 | Entropy loss: -0.0032  | Total Loss: 4.38 | Total Steps: 41
TEST: ---black sphere---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 3/100  | Episode Reward: 10  | Actor loss: 0.00 | Critic loss: 0.17 | Entropy loss: -0.0006  | Total Loss: 0.17 | Total Steps: 31
TEST: ---red capsule---
TEST: Decision Step reward: -2.5
TEST: Decision Step reward: -2.5
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 4/100  | Episode Reward: 5  | Actor loss: 0.00 | Critic loss: 0.06 | Entropy loss: -0.0029  | Total Loss: 0.06 | Total Steps: 43
TEST: --

...

Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3285/68000  | Episode Reward: 10  | Average Reward 9.85  | Actor loss: 0.22 | Critic loss: 0.21 | Entropy loss: -0.0028  | Total Loss: 0.42 | Total Steps: 39

---yellow sphere---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3286/68000  | Episode Reward: 10  | Average Reward 9.85  | Actor loss: -0.00 | Critic loss: 0.31 | Entropy loss: -0.0002  | Total Loss: 0.31 | Total Steps: 41

---yellow cube---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3287/68000  | Episode Reward: 10  | Average Reward 9.85  | Actor loss: 0.00 | Critic loss: 0.84 | Entropy loss: -0.0001  | Total Loss: 0.85 | Total Steps: 31

---black prism---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3288/68000  | Episode Reward: 8  | Average Reward 9.82  | Actor loss: -0.01 | Critic loss: 2.15 | Entropy loss: -0.0001  | Total Loss: 2.14 | Total Ste

TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 15/100  | Episode Reward: 10  | Average Reward 8.90  | Actor loss: 0.00 | Critic loss: 1.37 | Entropy loss: -0.0225  | Total Loss: 1.34 | Total Steps: 35
TEST: ---yellow prism---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 16/100  | Episode Reward: 10  | Average Reward 8.90  | Actor loss: 0.00 | Critic loss: 0.16 | Entropy loss: -0.0029  | Total Loss: 0.16 | Total Steps: 41
TEST: ---green cube---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 17/100  | Episode Reward: 10  | Average Reward 8.95  | Actor loss: 0.00 | Critic loss: 0.29 | Entropy loss: -0.0010  | Total Loss: 0.29 | Total Steps: 31
TEST: ---green cube---
TEST: Decision Step reward: -2.5
TEST: Decision Step reward: -2.5
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 18/100  | Episode Reward: 5  | Average Reward 8.90  | Actor loss: -0.

TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 45/100  | Episode Reward: 8  | Average Reward 9.03  | Actor loss: 0.01 | Critic loss: 0.09 | Entropy loss: -0.0019  | Total Loss: 0.10 | Total Steps: 29
TEST: ---yellow prism---
TEST: Decision Step reward: -2.5
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 46/100  | Episode Reward: 8  | Average Reward 9.00  | Actor loss: 0.00 | Critic loss: 0.74 | Entropy loss: -0.0149  | Total Loss: 0.72 | Total Steps: 41
TEST: ---yellow prism---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 47/100  | Episode Reward: 10  | Average Reward 9.05  | Actor loss: 0.05 | Critic loss: 7.46 | Entropy loss: -0.0216  | Total Loss: 7.49 | Total Steps: 66
TEST: ---yellow prism---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 48/100  | Episode Reward: 10  | Average Reward 9.07  | Actor loss: 0.14 | Critic loss: 0.31 | Entrop

TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 75/100  | Episode Reward: 10  | Average Reward 9.00  | Actor loss: 0.01 | Critic loss: 0.36 | Entropy loss: -0.0006  | Total Loss: 0.37 | Total Steps: 36
TEST: ---yellow prism---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 76/100  | Episode Reward: 10  | Average Reward 9.00  | Actor loss: 0.00 | Critic loss: 0.80 | Entropy loss: -0.0074  | Total Loss: 0.80 | Total Steps: 36
TEST: ---black sphere---
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 77/100  | Episode Reward: 10  | Average Reward 9.00  | Actor loss: 0.01 | Critic loss: 0.19 | Entropy loss: -0.0070  | Total Loss: 0.19 | Total Steps: 69
TEST: ---blue cylinder---
TEST: Decision Step reward: -2.5
TEST: Decision Step reward: -2.5
TEST: Step: 100
TEST: Decision Step reward: -2.5
TEST: Agent in terminal steps
TEST: Terminal Step reward: 10.0
Testing  | Episode: 78/100  | Episo

Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3307/68000  | Episode Reward: 10  | Average Reward 9.80  | Actor loss: 0.01 | Critic loss: 0.83 | Entropy loss: -0.0001  | Total Loss: 0.84 | Total Steps: 31

---black cylinder---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3308/68000  | Episode Reward: 10  | Average Reward 9.80  | Actor loss: -0.04 | Critic loss: 0.18 | Entropy loss: -0.0009  | Total Loss: 0.14 | Total Steps: 38

---red cube---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3309/68000  | Episode Reward: 10  | Average Reward 9.80  | Actor loss: 0.00 | Critic loss: 0.12 | Entropy loss: -0.0001  | Total Loss: 0.12 | Total Steps: 36

---blue prism---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3310/68000  | Episode Reward: 10  | Average Reward 9.80  | Actor loss: -0.00 | Critic loss: 0.13 | Entropy loss: -0.0003  | Total Loss: 0.12 | Total Steps: 37

---yellow capsule---


In [9]:

data = {
            'all_average_reward': all_average_reward,
            'all_episode_reward': all_episode_reward,
            'all_actor_loss': all_actor_loss,
            'all_critic_loss': all_critic_loss,
            'all_entropy_loss': all_entropy_loss,
            'all_total_loss': all_total_loss,
            'all_steps': all_steps,
        } 
file_path = f'result/{ALG_NAME}_{ENV_ID}_train.txt'
with open(file_path, 'w') as file:
    json.dump(data, file)

test_data = {
            'all_average_reward': test_average_reward,
            'all_episode_reward': test_episode_reward,
            'all_actor_loss': test_actor_loss,
            'all_critic_loss': test_critic_loss,
            'all_entropy_loss': test_entropy_loss,
            'all_total_loss': test_total_loss,
            'all_steps': test_steps,
        } 
file_path = f'result/{ALG_NAME}_{ENV_ID}_test.txt'
with open(file_path, 'w') as file:
    json.dump(test_data, file)

